In [1]:
from flask import Flask, request, jsonify
import json
import logging
import unittest
app = Flask(__name__)

In [2]:
# List of JSON objects (hardcoded for simplicity)
with open('hard_code.json', 'r') as file:
    json_data = json.load(file)

In [3]:
# Logger setup
logging.basicConfig(filename='backend.log', level=logging.INFO)


# Endpoint to get an object based on ID
@app.route('/get_object/<int:id>', methods=['GET'])
def get_object(id):
    for obj in json_data:
        if obj['id'] == id:
            return jsonify(obj)
    return jsonify({'message': 'Object not found'}), 404

In [4]:
# Endpoint to add a new object
@app.route('/add_object', methods=['POST'])
def add_object():
    data = request.json
    
    # Append new object to the list
    json_data.append(data)
    
    # Write updated JSON data back to file
    with open('data.json', 'w') as file:
        json.dump(json_data, file, indent=4)
    
    return jsonify({'message': 'Object added successfully'}), 201

In [5]:
# Endpoint to delete an object
@app.route('/delete_object/<int:id>', methods=['DELETE'])
def delete_object(id):
    
    # Find and remove the object with the specified ID
    for obj in json_data:
        if obj['id'] == id:
            json_data.remove(obj)
            # Write updated JSON data back to file
            with open('data.json', 'w') as file:
                json.dump(json_data, file, indent=4)
            return jsonify({'message': 'Object deleted successfully'})
    
    return jsonify({'message': 'Object not found'}), 404


In [6]:
# Dependency injection for processing added objects
class ObjectProcessor:
    def process(self, obj):
        # For simplicity, just printing the object here
        print("Processing object:", obj)

In [7]:
# Define your unit tests
class TestBackendAPI(unittest.TestCase):
    def setUp(self):
        # Create a test client
        self.app = app.test_client()
        # Propagate exceptions to the test client
        self.app.testing = True

    def test_get_object(self):
        logging.info("Starting test_get_object")
        # Test case for GET /get_object/<id>
        response = self.app.get('/get_object/2')
        data = response.get_json()
        self.assertEqual(response.status_code, 200)
        self.assertEqual(data['name'], 'Alice')
        self.assertEqual(data['age'], 25)
        logging.info("Finished test_get_object")

    def test_add_object(self):
        logging.info("Starting test_add_object")
        # Test case for POST /add_object
        new_object = {"id": 4, "name": "Sally", "age": 44}
        response = self.app.post('/add_object', json=new_object)
        self.assertEqual(response.status_code, 201)
        # Check if the object was added to the list
        self.assertIn(new_object, json_data)
        logging.info("Finished test_add_object")

    def test_delete_object(self):
        logging.info("Starting test_delete_object")
        # Test case for DELETE /delete_object/<id>
        response = self.app.delete('/delete_object/3')
        self.assertEqual(response.status_code, 200)
        # Check if the object was deleted from the list
        self.assertNotIn({"id": 3, "name": "Bob", "age": 35}, json_data)
        logging.info("Finished test_delete_object")
        
    def tearDown(self):
        # Clean up resources
        logging.shutdown()

# Run the unit tests
unittest.main(argv=[''], verbosity=2, exit=False)


test_add_object (__main__.TestBackendAPI.test_add_object) ... ok
test_delete_object (__main__.TestBackendAPI.test_delete_object) ... ok
test_get_object (__main__.TestBackendAPI.test_get_object) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.009s

OK


In [8]:
print(json_data)

[{'id': 1, 'name': 'John', 'age': 30}, {'id': 2, 'name': 'Alice', 'age': 25}, {'age': 44, 'id': 4, 'name': 'Sally'}]
